# **[HW8] Practice_6**
1. Install packages
2. PageRank

선형대수 실습은, exercise 혹은 coding 문제풀이에 집중하기 보다는, 수업시간에 배웠던 개념들을 visualize 하면서 이를 폭넓게 이해하는 데에 초점을 맞추고 있습니다. 실습에서 사용한 예시 외에도, 다양한 matrix들을 visualize 하면서 개념을 익혀보시기 바랍니다.

# 1. Install packages

> 필요한 package를 설치하고 import합니다

In [1]:
# visualization을 위한 helper code입니다.
from urllib.request import urlretrieve
URL = 'https://go.gwu.edu/engcomp4plot'
urlretrieve(URL, 'plot_helper.py')

import sys
sys.path.append('../scripts/')

# 다음 세 custom function (1)plot_vector, (2)plot_linear_transformation, (3) plot_linear_transformations
# 을 사용할 것입니다.
from plot_helper import *

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import scipy as sp
import scipy.linalg
import sympy as sy

sy.init_printing() 
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

# 2. PageRank

> 이번 실습에선 Google의 PageRank 알고리즘을 구현하면서, eigenvector가 실제로 어떻게 사용되는지 알아보도록 하겠습니다.

> PageRank는 Google의 검색 알고리즘 중 가장 먼저 사용되었던 알고리즘입니다. PageRank는 web**Page**의 **rank**를 매기기 위해, 해당 webpage와 연결되어 있는 다른 webpage들의 개수가 몇 개인지,  (ii) 연결된 webpage들은 각각 얼마나 중요한 webpage인지, 두 가지를 고려하게 됩니다. PageRank에 깔려있는 가정은, **더 중요한 webpage 일수록, 다른 중요한 webpage들로부터 link가 걸려있을 가능성이 높다**는 것입니다 (https://en.wikipedia.org/wiki/PageRank).

> 어떤 webpage의 점수는, 해당 webpage를 link 건 다른 webpage들의 중요도의 weighted average로 계산합니다. 다음 예시를 통해서 알아봅시다.

<center>
<img src="https://nbviewer.jupyter.org/github/engineersCode/EngComp4_landlinear/blob/master/images/PR-4pages.png
" width="400" height="300"> <figcaption>Example network of 4 webpages, 출처: Ref. [1]</figcaption>
</center>


> 위 그림에선, 4개의 wegbpage가 서로 link를 걸고 있음을 알 수 있습니다 (화살표가 링크를 걸었다는 의미입니다). D의 rank(중요도)는 몇일까요? 위 그림에선, D를 link 건 webpage가 A, C입니다. 따라서 A와 C는 D에게 본인의 rank를 나눠주게 됩니다. 이때 A는 나머지 3개 webpage를 전부 link걸었기 때문에, B,C,D 세 webpage에 1/3씩 본인의 rank를 나눠주게 됩니다. B는 B,C 두 webpage를 link걸었기 때문에, B,C 두 webpage에 1/2씩 본인의 rank를 나눠주게 됩니다. 

* Page A : B,C,D에게 본인 rank의 1/3씩 나눠주게 됨
* Page B : C,D에게 본인 rank의 1/2씩 나눠주게 됨
* Page C : Link가 없음
* Page D : A,C에게 본인 rank의 1/2씩 나눠주게 됨

> 위 내용을 행렬로 정리하면, 다음과 같이 표현할 수 있습니다.

$$M = \begin{bmatrix} 0   & 0   & 0 & 1/2 \\
                       1/3 & 0   & 0 & 0  \\
                       1/3 & 1/2 & 0 & 1/2 \\
                       1/3 & 1/2 & 0 & 0 \end{bmatrix}
$$

> 그럼 최종적으로 A,B,C,D의 rank는 어떻게 계산될까요? 최종 rank는 모든 webpage들의 rank가 변하지 않을 때까지 나눠주는 행위를 반복한 다음, 수렴된 값을 최종 rank로 생각하게 됩니다.

$$
\begin{bmatrix} 1/4 \\ 1/4 \\ 1/4 \\ 1/4 \end{bmatrix}
\xrightarrow{M} Mx=\begin{bmatrix} 1/8 \\ 1/12 \\ 1/3 \\ 5/24 \end{bmatrix}\xrightarrow{M} M^2x = \begin{bmatrix} 5/48 \\ 1/24 \\ 3/16 \\ 1/12 \end{bmatrix} \xrightarrow{M} \cdots \xrightarrow{M} M^{n}x \xrightarrow{M} M^{n+1}x
$$

> $M^nx = M^{n+1}x$가 된다면, 더이상 값이 바뀌지 않는다는 의미이고, 그 값이 A,B,C,D 각각의 최종 rank가 됩니다.

> 그런데 문제점이 있습니다. Webpage C는 다른 webpage로 link를 걸기 때문에, rank를 나눠주지 않고 webpage C의 rank는 소멸되어버리기 때문에, 최종적으로 모든 webpage의 rank가 0으로 되어버리게 됩니다 (위의 예시에서 모든 rank의 합이 1, 0.75, 0.4167, ... 으로 점점 주는 것을 알 수 있습니다).

In [3]:
M = np.array([[0,0,0,1/2],[1/3,0,0,0],[1/3,1/2,0,1/2],[1/3,1/2,0,0]])
x = np.array([1/4,1/4,1/4,1/4])

In [4]:
print(M@x)
print(M@M@M@M@M@M@M@M@M@M@M@M@M@M@x)

[0.125 0.083 0.333 0.208]
[0. 0. 0. 0.]


> 따라서, 이 문제를 해결하기 위해 $M$을 약간 수정해야 합니다. 가장 단순한 해결책은, M의 zero-column을 $\begin{bmatrix} 1/4 \\ 1/4 \\ 1/4 \\ 1/4 \end{bmatrix}$로 바꿔버리는 것입니다. 이를 modified matrix, $M'$이라고 합시다. 최종적으로는, 이 $M'$과 모든 entry가 1/4인 행렬과의 가중 평균을 사용하게 됩니다. 이 모든 entry가 1/4인 행렬을 더해주게 되면, 모든 entry가 0 이상이 되고, 이는 어떤 임의의 webpage에서 본인을 포함한 다른 webpage로 향하는 모든 link를 만들어줬다고 생각할 수 있습니다.

$$
  M' = \begin{bmatrix} 0   & 0   & 1/4 & 1/2 \\
                      1/3 & 0   & 1/4 & 0  \\
                      1/3 & 1/2 & 1/4 & 1/2 \\
                      1/3 & 1/2 & 1/4 & 0 \end{bmatrix}
\\
  B = n \times n \text{ matrix with 1/n in every entry} \\
  G = (1-p)\, M^{\prime} + p\, B
$$

> $p$는 *damping factor*인데, 둘 간의 가중치를 정하기 위해 우리가 지정하는 hyper-parameter입니다. 결론적으로는, $G$는 $M$과 다르게 $x$과 non-zero vector로 수렴하게 됩니다. 이를 좀 더 엄밀하게 알고 싶으신 분은, 다음 강의를 참조하시기 바랍니다 (https://ocw.mit.edu/courses/mathematics/18-06-linear-algebra-spring-2010/video-lectures/lecture-24-markov-matrices-fourier-series/.)

### Exercise

In [5]:
# answer
p=0.15

modified_M = M
modified_M[:,2] = np.ones(4)*1/4
B = np.ones((4,4))*1/4
G = 0.85*modified_M + 0.15*B

> 위 내용을 참조하여, PageRank algorithm을 구현해 봅시다. Parameter로 $M'$, damping factor $p$, tolerance $\epsilon$를 받아서, 최종 수렴된 vector를 return하면 됩니다. 위 설명처럼 $G^nx=G^{n+1}x$가 됐다면 rank vector가 수렴되었음을 의미하는데, 둘의 차가 정확히는 같지 않더라도 $\epsilon$ 이하라면 수렴되었다고 간주합니다.

$$ ||G^{n+1}x - G^nx||^2_2 < \epsilon \rightarrow \text{ rank vector is converged to }G^nx $$

### Exercise

In [6]:
# answer
def pagerank(modified_M, p=0.15, eps=1e-3):

    n = modified_M.shape[1]
    v = np.random.rand(n)
    v = v/np.linalg.norm(v)
    G = (1-p)*modified_M + p*(1/n)*np.ones((n,n))
    vold = np.zeros(n)
    
    while np.linalg.norm(v-vold, 2)>eps:
        vold = v.copy()
        v = G.dot(v)
        
    return G, v/np.sum(v)

In [7]:
pagerank(modified_M)

(array([[0.037, 0.037, 0.25 , 0.462],
        [0.321, 0.037, 0.25 , 0.037],
        [0.321, 0.462, 0.25 , 0.462],
        [0.321, 0.462, 0.25 , 0.037]]), array([0.219, 0.175, 0.356, 0.25 ]))

> 실제로 G를 여러번 곱했을 때, 해당 값으로 수렴함을 확인할 수 있습니다.

In [8]:
G@G@G@G@G@G@G@G@G@G@G@G@G@G@G@G@G@G@G@G@x

array([0.219, 0.175, 0.356, 0.25 ])

> 즉 세 번쨰, C가 가장 높은 rank를 획득했음을 알 수 있습니다. B는 가장 낮은 rank를 획득하였습니다. 각 rank는 임의의 user가 각 page에 도달할 확률이라고 해석할 수 있습니다.

### Exercise

> 다음 network에 pagerank 알고리즘을 적용하여 최종 rank vector를 구하시오 (damping factor $p$=0.15).

<center>
<img src="https://github.com/engineersCode/EngComp4_landlinear/raw/5a11cb4153cc51833f524b803a4cd185966cd1fb/images/google-irreducible.png
" width="400" height="300"> <figcaption>Example network of 8 webpages, 출처: Ref. [2]</figcaption>
</center>



In [9]:
# answer

K1 = np.array([[0,1/2,1/2,0,0,0,0,0],[0,0,0,1,0,0,0,0],[0,1/2,0,0,1/2,0,0,0],[0,1/2,0,0,0,1/2,0,0],[0,0,0,0,0,1/3,1/3,1/3],
              [0,0,0,0,0,0,0,1],[1/3,0,0,0,1/3,0,0,1/3],[0,0,0,0,0,1/2,1/2,0]])
K1 = K1.T

K2, v2 = pagerank(K1)
v2

array([0.06 , 0.112, 0.044, 0.114, 0.079, 0.195, 0.147, 0.249])

> 위와 같이 수렴된 vector는, 사실 eigenvector를 이용하면 쉽게 풀 수 있습니다. 첫 번쨰 예시 G에서는 $G^nx = G^{n+1}x$을 만족하는 $G^nx$를 찾으면 됐습니다. 이때 $G^nx = v$라고 하면, 이는 $v = Gv$가 됩니다. 즉, $v$는 $G$의 eigenvalue가 1인 eigenvector가 됩니다.

In [10]:
eigenvalues, eigenvectors = np.linalg.eig(K2)
eigenvalues

array([ 1.   +0.j   , -0.763+0.j   ,  0.397+0.154j,  0.397-0.154j,
       -0.551+0.j   , -0.164+0.204j, -0.164-0.204j,  0.   +0.j   ])

> 이 중 eigenvalue가 1인, 첫번째 eigenvector는 위에서 구한 rank vector와 같음을 알 수 있습니다.


In [11]:
eigenvectors

array([[ 0.152+0.j   , -0.165+0.j   , -0.205+0.065j, -0.205-0.065j,
         0.27 +0.j   ,  0.038-0.225j,  0.038+0.225j, -0.   +0.j   ],
       [ 0.28 +0.j   ,  0.108+0.j   ,  0.276+0.107j,  0.276-0.107j,
         0.253+0.j   , -0.051+0.067j, -0.051-0.067j,  0.   +0.j   ],
       [ 0.111+0.j   ,  0.092+0.j   , -0.167+0.135j, -0.167-0.135j,
        -0.208+0.j   , -0.323+0.18j , -0.323-0.18j ,  0.   +0.j   ],
       [ 0.285+0.j   , -0.12 +0.j   ,  0.591+0.j   ,  0.591-0.j   ,
        -0.39 +0.j   ,  0.273-0.006j,  0.273+0.006j, -0.   +0.j   ],
       [ 0.199+0.j   , -0.217+0.j   , -0.312+0.251j, -0.312-0.251j,
         0.43 +0.j   ,  0.595+0.j   ,  0.595-0.j   ,  0.802+0.j   ],
       [ 0.49 +0.j   ,  0.512+0.j   ,  0.227-0.234j,  0.227+0.234j,
        -0.224+0.j   , -0.145-0.18j , -0.145+0.18j , -0.267+0.j   ],
       [ 0.369+0.j   ,  0.446+0.j   , -0.323-0.021j, -0.323+0.021j,
        -0.525+0.j   ,  0.14 +0.158j,  0.14 -0.158j, -0.   +0.j   ],
       [ 0.624+0.j   , -0.656+0.j   , -0.

In [12]:
rank_vector = K2 @ eigenvectors[:,0]
rank_vector = rank_vector / rank_vector.sum()
rank_vector

array([0.06 +0.j, 0.112+0.j, 0.044+0.j, 0.114+0.j, 0.079+0.j, 0.195+0.j,
       0.147+0.j, 0.249+0.j])

> j는 허수 부분을 나타내는데, 계수가 0이므로 무시하면 됩니다. 즉, $G$의 eigenvalue가 1인 eigenvector는 $G$를 여러번 곱해서 얻어진 rank_vector와 같음을 알 수 있습니다.

# References

1. Dan Margalit and Joseph Rabinoff (2018), Interactive Linear Algebra, an open book under the GNU Free Documentation License, Version 1.2.
2. David Austin (2019), Understanding Linear Algebra, an open book under CC-By license.
3. https://github.com/jclosure/EngComp4_landlinear